## LAB6 : Sephora Data Scraping 

### The aim of this lab is to understand the concept and importance of webscraping by performing some aspect of it on Sephora website

#### Over here, we plan to target the skin care categories of sephora and our code uses webscraping to collect the URL related to every skin care product there is.

#### Installing BeautifulSoup:

In [104]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


#### Importing:

In [105]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [106]:
#A function that gets the browser to scroll down:
#ref:  https://www.hackerearth.com/fr/practice/notes/praveen97uma/crawling-a-website-that-loads-content-using-javascript-with-selenium-webdriver-in-python/

def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

In [107]:

chrome_path = "C:/Users/aishwarya/Downloads/chromedriver_win32/chromedriver"
driver = webdriver.Chrome(executable_path = chrome_path)


#webscrapig with respect to the different skin care categories that Sephora has:
categories = ['moisturizing-cream-oils-mists']            #Only one category is used for now, so that there is no excessive delay                                                                                                    
             #'cleanser'                                  #in the speed of the execution
             #'facial-treatments',
             #'eye-treatment-dark-circle-treatment',
             #'facial-treatment-masks',
            # 'sunscreen-sun-protection',
             #'lip-treatments']
                
#creating an empty data frame that all the URLs and categories will go in to
final_df = pd.DataFrame(columns = ['category', 'URL'])


for category in categories: #looping through the different categories

    page_num = 1 #the first page
    
    while page_num<5: # I have considered to keep page_num <5 to expedite the execution
        url = 'https://www.sephora.com/shop/'+ category + '?pageSize=300&currentPage=' + str(page_num)
        driver.get(url)
        time.sleep(5)
        try:   
            if driver.find_element_by_class_name('css-3a7b4s').is_displayed():
                break
            
        except:
    
         
            try:
            
                xpath = '//*[@id="modalDialog"]/button'
                btn = driver.find_element_by_xpath(xpath)
                btn.click()
                time.sleep(20)
            except:
                pass
    
           
            old_len = 0
            while True:
                browser = scrollDown(driver, 20) #scroll down the page
                time.sleep(10) #give it time to load
                slug = driver.find_elements_by_class_name('css-ix8km1') #look for the urls of products
                new_len = len(slug)
                if old_len == new_len: #if the old length and new length are equal, the bottom of page was reached
                    break
                else:
                    old_len = new_len

            #from the list of URLs in slug pull all the href and make a dictionary with it and the category name        
            slugURL = []
            for a in slug:
                subURL = {}
                subURL['category'] = category 
                subURL['URL'] = a.get_attribute('href')
                slugURL.append(subURL)
    
            #append our data frame with categories and URLs
            df = pd.DataFrame(slugURL)
            #print(df.head())
            #adding to go to next page
            page_num += 1
            #concatenating to get all in same df
            final_df = pd.concat([final_df, df], axis = 0, ignore_index = True) 
    
#close the chrome
driver.close()
            

In [108]:
#checking the structure of my final dataframe
final_df

,category,URL
0,moisturizing-cream-oils-mists,https://www.sephora.com/product/protini-tm-pol...
1,moisturizing-cream-oils-mists,https://www.sephora.com/product/glow-recipe-wa...
2,moisturizing-cream-oils-mists,https://www.sephora.com/product/cicapair-tiger...
3,moisturizing-cream-oils-mists,https://www.sephora.com/product/the-dewy-skin-...
4,moisturizing-cream-oils-mists,https://www.sephora.com/product/barrier-lipid-...
...,...,...
472,moisturizing-cream-oils-mists,https://www.sephora.com/product/resilience-lif...
473,moisturizing-cream-oils-mists,https://www.sephora.com/product/blue-herbal-mo...
474,moisturizing-cream-oils-mists,https://www.sephora.com/product/sleep-face-bod...
475,moisturizing-cream-oils-mists,https://www.sephora.com/product/boosted-contou...


In [93]:

#saving the final dataframe to a csv folder
final_df.to_csv('product_urls.csv', index = False)

#### These URLs can now be used to gather important data aboout each skin-care product.

# Data Preprocesing:

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt


In [95]:
# step 1. cleaning data
cosm = pd.read_csv('product_urls.csv')


In [ ]:
#making the category names more understandable:
cosm.Category[cosm['Category'] == 'moisturizing-cream-oils-mists'] = str('Moisturizer')
cosm.Category[cosm['Category'] == 'cleanser'] = str('Cleanser')
cosm.Category[cosm['Category'] == 'facial-treatments'] = str('Treatment')
cosm.Category[cosm['Category']== 'face-mask'] = str('Face Mask')
cosm.Category[cosm['Category'] == 'eye-treatment-dark-circle-treatment'] = str('Eye cream')
cosm.Category[cosm['Category']== 'sunscreen-sun-protection'] = str('Sun protect')

#### Data preprocessing can contain many other ways like checking for null values, removing duplictae values, tokenize an ingredient list etc.

#### Data preprocessing is done so that the data is ready for what lies ahead of it